In [66]:
# Import pour la génération aléatoire
import random as random
# Import pour l'affichage uniquement
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
# Import pour les tests de mesure temporelle
import time

In [86]:
class Generateur(object):
    """ Générateur de mot plausible implémentant un dictionnaire de dictionnaire
    """

    def __init__(self,profondeur):
        self.nbAppear = dict() # Initialise un dictionnaire vide
        self.N = profondeur # Initialise une profondeur
    
    def ajout(self, arbre, caracteres):
        """ Incrémente le nombre d'occurence d'une lettre, le créé si non présent
        """
        if len(caracteres)==1 : # On vérifie qu'on est à la bonne profondeur
            if caracteres[0] in arbre : # On vérifie la présence du  caractère
                arbre[caracteres[0]] += 1 # On l'incrémente s'il existe
            else :
                arbre[caracteres[0]] = 1 # On créé l'entrée sinon
        else :
            if caracteres[0] in arbre : # On vérifie la présence du caractère 
                self.ajout(arbre[caracteres[0]], caracteres[1:])
            else :
                arbre[caracteres[0]] = dict() # On créé le sous dictionnaire avec l'entrée
                self.ajout(arbre[caracteres[0]], caracteres[1:])
        
    def lireMot(self, mot):
        """ Lit un mot et dénombre les occurrences de ses couples de lettre
        """
        for i in range(len(mot)-(self.N-1)): # On parcourt le mot
            caracteres = []
            for y in range(self.N):
                caracteres.append(mot[i+y])
            self.ajout(self.nbAppear, caracteres) # On incrémente le nombre d'occurrence de la lettre
    
    def _estLettre(self, c):
        """ Détermine si un caractère donné est une lettre
        """
        return ("a" <= c <= "z") or ("à" <= c <= "ü")

    def __trouveMot(self,texte,i):
        """ Trouve le premier mot dans une chaîne donnée à partir d'un indice donné.
            Renvoi le mot et l'indice de fin
        """
        mot = " "*(self.N-1) # On démarre par le caractère de début de mot
        while self._estLettre(texte[i].lower()) and i < len(texte): # On parcourt tant qu'on a une lettre
            mot+=texte[i].lower() # On ajoute la lettre au mot
            i+=1
        return mot+" ", i+1 # On ajoute une espace comme caractère de fin de mot

    def lireTexte(self,fichier):
        """ Lit un fichier texte dont le nom est donné en paramètre pour nourrir le dictionnaire
        """
        texte = open("./ProjetPython/"+fichier, "r").read()
        i = 0
        while i < len(texte):
            mot, i = self.__trouveMot(texte,i) # On récupère les mots
            if mot != " "*self.N : # On nourri le dictionnaire si le mot n'est pas vide
                self.lireMot(mot)
    
    
    def __trouveLettre(self, caracteres):
        """ Fait la somme des occurrences des lettres possibles suivant une liste de lettres donnée 
            et fait un tirage uniforme sur cette somme
        """
        somme = 0
        arbre = self.nbAppear
        for c in caracteres :
            arbre = arbre[c]
        
        for occurrence in arbre.values(): # Pour toutes les occurrences dans le dictionnaire d'une lettre
            somme += occurrence # On l'ajoute à la somme
        
        return random.randint(0,somme) # On renvoi une valeur aléatoire entre 0 et la somme
        
    def caractereSuivant(self, caracteres):
        """ Prend une liste de caractère et tire au sort le caractère suivant
        """
        alea = self.__trouveLettre(caracteres) # On fait un tirage aléatoire
        i = 0
        arbre = self.nbAppear
        for c in caracteres :
            arbre = arbre[c]
            
        occurences = list(arbre.values())
        while alea > occurences[i]: # Si le tirage est plus grand que le nombre d'occurence d'une lettre
            alea-= occurences[i] # On soustrait au tirage le nombre d'occurences de cette lettre
            i+=1
        return list(arbre.keys())[i] # On renvoi la lettre tiré
    
    def genereMot(self, premiereLettre=' '):
        """ Génère un mot commançant par une lettre donnée. Trouve la première lettre suivant le caractère de début de mot si non spécifié
        """
        mot = ' '*(self.N-2) + premiereLettre
        if len(mot)==(self.N-1) :
            mot+=self.caractereSuivant(mot[0-(self.N-1):])
        while(mot[-1]!=' ') : # Tant que la dernière lettre n'est pas le caractère de fin de mot ou que le mot est vide
            mot+=self.caractereSuivant(mot[0-(self.N-1):]) # On ajoute la lettre suivante en fonction de la dernière lettre actuelle
        return mot[(self.N-1):-1] if mot[self.N-2]==' ' else mot[(self.N-2):-1] # Renvoi la mot, sans espace si première lettre non spécifiée
    
    
    def afficheMatrice(self):
        """ Génère la matrice de transition et l'affiche
        """
        ToutesLettres = sorted([key for key in self.nbAppear.keys()]) # On récupère toute les lettres
        # On génère une matrice carrée numpy en fonction du nombre de lettre
        prop = np.matrix([[0. for y in range(0,len(ToutesLettres))] for x in range(0,len(ToutesLettres))])
        
        
        # On remplit cette matrice avec les occurrences dans le dictionnaire
        i = 0
        for letter1 in ToutesLettres:
            y = 0
            for letter2 in ToutesLettres:
                if letter2 in self.nbAppear[letter1].keys():
                    prop[i,y] = self.nbAppear[letter1][letter2]
                y+=1
            i+=1
        
        # Transforme les occurences en pourcentage
        Somm = [x.sum() for x in prop]
        for i in range(0,len(Somm)) :
            for y in range(0,len(Somm)) :
                prop[i,y] = (float(prop[i,y])*100.0)/float(Somm[i])
        
        # Affiche la matrice
        fig, ax = plt.subplots(figsize=(7,7))
        im = ax.imshow(prop, cmap=cm.binary)

        ax.set_xticks(np.arange(len(ToutesLettres)))
        ax.set_yticks(np.arange(len(ToutesLettres)))

        ax.set_xticklabels(ToutesLettres)
        ax.set_yticklabels(ToutesLettres)

        ax.set_title("Probability of letters")
        fig.tight_layout()
        plt.show()


In [87]:
N1 = 2
N2 = 10

g1 = Generateur(N1)
g2 = Generateur(N2)

g3 = Generateur(N1)
g4 = Generateur(N2)

débutLectureG1 = time.time()
g1.lireTexte("notredame.txt")
finG1DébutG2 = time.time()
g2.lireTexte("notredame.txt")
finG2DébutG3 = time.time()
g3.lireTexte("littlewomen.txt")
finG3DébutG4 = time.time()
g4.lireTexte("littlewomen.txt")
finLectureG4 = time.time()
print(" Temps de construction G1 : ", '%.3f'%(finG1DébutG2 - débutLectureG1),'sec')
print(" Temps de construction G2 : ", '%.3f'%(finG2DébutG3 - finG1DébutG2),'sec')
print(" Temps de construction G3 : ", '%.3f'%(finG3DébutG4 - finG2DébutG3),'sec')
print(" Temps de construction G4 : ", '%.3f'%(finLectureG4 - finG3DébutG4),'sec')


#g1.afficheMatrice()
#g3.afficheMatrice()

 Temps de construction G1 :  2.580 sec
 Temps de construction G2 :  7.209 sec
 Temps de construction G3 :  2.525 sec
 Temps de construction G4 :  6.894 sec


In [90]:
print("Notre Dame :")
print("  N={0:35} N={1:3}".format(str(N1),str(N2)))
débutGeneration = time.time()
for i in range(10):
    mot1 = g1.genereMot()
    mot2 = g2.genereMot()
    print("  Mot aléatoire : {0:20}  Mot aléatoire : {1:20}".format(mot1,mot2))

finGeneration = time.time()
print(" Temps de génération : ", '%.4f'%(finGeneration - débutGeneration),"sec")

print("")
print("Little Women :")
print("  N={0:35} N={1:3}".format(str(N1),str(N2)))
débutGeneration = time.time()
for i in range(10):
    mot1 = g3.genereMot()
    mot2 = g4.genereMot()
    print("  Mot aléatoire : {0:20}  Mot aléatoire : {1:20}".format(mot1,mot2))

finGeneration = time.time()
print(" Temps de génération : ", '%.4f'%(finGeneration - débutGeneration),"sec")

Notre Dame :
  N=2                                   N=10 
  Mot aléatoire : flarm                 Mot aléatoire : enceinte            
  Mot aléatoire : nn                    Mot aléatoire : devinait            
  Mot aléatoire : ce                    Mot aléatoire : manqué              
  Mot aléatoire : ue                    Mot aléatoire : murmura             
  Mot aléatoire : laidourturqures       Mot aléatoire : ont                 
  Mot aléatoire : i                     Mot aléatoire : je                  
  Mot aléatoire : vible                 Mot aléatoire : le                  
  Mot aléatoire : be                    Mot aléatoire : charpente           
  Mot aléatoire : sillodeban            Mot aléatoire : mourir              
  Mot aléatoire : mpâcaîn               Mot aléatoire : zèle                
 Temps de génération :  0.0048 sec

Little Women :
  N=2                                   N=10 
  Mot aléatoire : t                     Mot aléatoire : jackdaw           